# Download Historical Streamflow Data

This Jupyter Notebook serves as the dedicated module for retrieving historical streamflow data, a pivotal component for the calibration and evaluation of the hydrological models in this study. Leveraging the capabilities of the `dataRetrieval` R package, designed for accessing U.S. Geological Survey (USGS) water data, the notebook systematically downloads streamflow records for all active and relevant stream gauge stations identified within the previously defined study region. The acquired streamflow time series represent the observed hydrographs, which are indispensable for calibrating the ModClark model's parameters ($T_c$ and $R$) and for rigorously assessing the predictive performance of the machine learning models by comparing their generated hydrographs against these real-world observations.

### Load the streamflow station data

In [1]:
# Load required packages
library(sf)           
library(dataRetrieval) 
library(dplyr) 
library(arrow)
library(IRdisplay)      

Linking to GEOS 3.8.1, GDAL 3.2.1, PROJ 7.2.1


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'arrow'


The following object is masked from 'package:utils':

    timestamp




In [2]:
# Load the .gpkg file
gpkg_file <- "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/geo/gpkg/study_area_stream_gauge_stn.gpkg"
stations <- st_read(gpkg_file)
glimpse(stations)

Reading layer `study_area_stream_gauge_stn' from data source 
  `/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/silver/geo/gpkg/study_area_stream_gauge_stn.gpkg' 
  using driver `GPKG'
Simple feature collection with 1056 features and 14 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -89.16257 ymin: 35.70829 xmax: -78.16294 ymax: 42.15683
Geodetic CRS:  NAD83
Rows: 1,056
Columns: 15
$ STAID      <chr> "03165000", "03164000", "03165500", "03161000", "03208500",~
$ STANAME    <chr> "CHESTNUT CREEK AT GALAX, VA", "NEW RIVER NEAR GALAX, VA", ~
$ ST         <chr> "va", "va", "va", "nc", "va", "va", "va", "ky", "va", "va",~
$ HUC        <chr> "05050001", "05050001", "05050001", "05050001", "05070202",~
$ CLASS      <int> 5, 0, 6, 5, 7, 8, 0, 0, 6, 6, 6, 0, 6, 7, 7, 0, 0, 0, 7, 0,~
$ FLOW       <dbl> 59.4, 0.0, 3690.0, 429.0, 1490.0, 2340.0, 0.0, 0.0, 5550.0,~
$ STAGE      <dbl> 1.56, 2.18, 4.26, 2.48, 4.60, 6.04, 11.11, 10.01, 3.05, 2.9~
$ TIME       <chr> "2

### Download Streamflow Data

The time span chosen for this study is 10 years. Therefore data will be downloaded from **01-01-2014 to 05-08-2025**.

In [3]:
# Define function to download 15-minute streamflow data
download_usgs_streamflow <- function(site_numbers, start_date, end_date, parameter_cd = "00060") {
  # Input validation
  if (!is.character(site_numbers) || length(site_numbers) == 0) {
    stop("site_numbers must be a non-empty character vector of USGS site IDs")
  }
  if (!is.character(start_date) || !is.character(end_date)) {
    stop("start_date and end_date must be character strings in YYYY-MM-DD format")
  }

  # Download 15-minute streamflow data using dataRetrieval
  # parameter_cd = "00060" is for discharge (cubic feet per second)
  streamflow_data <- readNWISuv(
    siteNumbers = site_numbers,
    parameterCd = parameter_cd,
    startDate = start_date,
    endDate = end_date,
    tz = "UTC"  # Time zone for data retrieval
  )

  # Check if data was retrieved
  if (nrow(streamflow_data) == 0) {
    # warning("No data retrieved for the specified sites and date range")
    return(NULL)
  }

  # Clean and format the data
  streamflow_data <- streamflow_data %>%
    select(site_no, dateTime, X_00060_00000, X_00060_00000_cd) %>%
    rename(
      discharge_cfs = X_00060_00000,
      discharge_quality = X_00060_00000_cd
    )
  
  return(streamflow_data)
}


In [4]:
# Define function to save data locally as Parquet
save_streamflow_data <- function(data, site_no, output_dir) {
  # Drop redundant column stite_no (this number is incorporated in the file name)
  # Create output directory if it doesn't exist
  if (!dir.exists(output_dir)) {
    dir.create(output_dir, recursive = TRUE)
  }
  
  # Check if data is valid
  if (is.null(data) || nrow(data) == 0) {
    # warning(paste("No data to save for site", site_no))
    return(FALSE)
  }
  
  # Define file path for Parquet
  file_path <- file.path(output_dir, paste0("streamflow_", site_no, ".parquet"))
  
  # Save data as Parquet
  write_parquet(data, file_path)
  # print(paste("Data saved for site", site_no, "at", file_path))
  return(TRUE)
}

**Download one station data as example**

In [5]:
station_code_example <- stations$STAID[1]

# Define date range
start_date <- "2014-01-01"
end_date <- "2025-05-08"

# Download data example
print(paste("Downloading data for station ID", station_code_example))
streamflow_data_example <- download_usgs_streamflow(
    site_numbers = station_code_example,
    start_date = start_date,
    end_date = end_date
)
glimpse(streamflow_data_example)
head(streamflow_data_example)


[1] "Downloading data for station ID 03165000"


GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=03165000&format=waterml,1.1&ParameterCd=00060&startDT=2014-01-01&endDT=2025-05-08



Rows: 388,238
Columns: 4
$ site_no           <chr> "03165000", "03165000", "03165000", "03165000", "031~
$ dateTime          <dttm> 2014-01-14 05:00:00, 2014-01-14 05:15:00, 2014-01-1~
$ discharge_cfs     <dbl> 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101.0, 101~
$ discharge_quality <chr> "A", "A", "A", "A", "A", "A", "A", "A", "A", "A", "A~


,site_no,dateTime,discharge_cfs,discharge_quality
,<chr>,<dttm>,<dbl>,<chr>
1,03165000,2014-01-14 05:00:00,101,A
2,03165000,2014-01-14 05:15:00,101,A
3,03165000,2014-01-14 05:30:00,101,A
4,03165000,2014-01-14 05:45:00,101,A
5,03165000,2014-01-14 06:00:00,101,A
6,03165000,2014-01-14 06:15:00,101,A


In [6]:
summary(streamflow_data_example)

   site_no             dateTime                   discharge_cfs    
 Length:388238      Min.   :2014-01-14 05:00:00   Min.   :  21.40  
 Class :character   1st Qu.:2016-12-22 07:18:45   1st Qu.:  45.20  
 Mode  :character   Median :2019-10-10 06:22:30   Median :  61.00  
                    Mean   :2019-10-04 06:20:27   Mean   :  83.63  
                    3rd Qu.:2022-07-20 15:56:15   3rd Qu.:  87.20  
                    Max.   :2025-05-09 03:45:00   Max.   :6230.00  
                                                  NA's   :576      
 discharge_quality 
 Length:388238     
 Class :character  
 Mode  :character  
                   
                   
                   
                   

**Download data for over 1000 USGS station within the study area**

In [5]:
# Download data for each station using a for loop
station_codes <- stations$STAID

# Define date range for 10 years
start_date <- "2014-01-01"
end_date <- "2025-05-08"

# Define output directory
output_dir <- "/Users/alan/Data Science Projects/ML-ModClark-IUH-Model/data/bronze/tabular/streamflow"

# Initialize lists to store stations with errors and error messages
error_stations <- character()
error_messages <- character()

count <- 0
len <- length(station_codes)

# Loop through each station code
for (site in station_codes) {
  count <- count + 1
  
  # Check if file already exists
  file_path <- file.path(output_dir, paste0("streamflow_", site, ".parquet"))
  if (file.exists(file_path)) {
    message(paste("File already exists for site", site, "- skipping download"))
    next
  }

  # Update status in notebook (overwrites previous output)
  display(paste0(count, "/", len, " - Downloading data for site ", site), display_id = "status", update = TRUE)
  
  
  # Try downloading data with error handling
  tryCatch({
    streamflow_data <- download_usgs_streamflow(
      site_numbers = site,
      start_date = start_date,
      end_date = end_date
    )
    
    # Save the data if download was successful
    if (!is.null(streamflow_data)) {
      save_streamflow_data(streamflow_data, site, output_dir)
    } else {
      warning(paste("No data available for site", site))
      error_stations <- c(error_stations, site)
      error_messages <- c(error_messages, "No data available")
    }
    
    # Add a delay to avoid overwhelming the USGS server
    Sys.sleep(1)
  }, error = function(e) {
    warning(paste("Error downloading data for site", site, ":", e$message))
    error_stations <- c(error_stations, site)
    error_messages <- c(error_messages, e$message)
  })
}

# Clear the status display
display("", display_id = "status", update = TRUE)

# Summary
message("Download and save process completed.")
message(paste("Check output directory:", output_dir))
if (length(error_stations) > 0) {
  message("Stations with errors: ", paste(error_stations, collapse = ", "))
  message("Error messages: ", paste(error_messages, collapse = "; "))
}

File already exists for site 03165000 - skipping download

File already exists for site 03164000 - skipping download

File already exists for site 03165500 - skipping download

File already exists for site 03161000 - skipping download

File already exists for site 03208500 - skipping download

File already exists for site 03209000 - skipping download

File already exists for site 03209200 - skipping download

File already exists for site 03209410 - skipping download

File already exists for site 03168000 - skipping download

File already exists for site 03167000 - skipping download

File already exists for site 03170000 - skipping download



[1] "12/1056 - Downloading data for site 03170500"

GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=03170500&format=waterml,1.1&ParameterCd=00060&startDT=2014-01-01&endDT=2025-05-08

Warning message in doTryCatch(return(expr), name, parentenv, handler):
"No data available for site 03170500"
File already exists for site 03171000 - skipping download

File already exists for site 03173000 - skipping download

File already exists for site 03175500 - skipping download



[1] "16/1056 - Downloading data for site 0317154954"

GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=0317154954&format=waterml,1.1&ParameterCd=00060&startDT=2014-01-01&endDT=2025-05-08

Warning message in doTryCatch(return(expr), name, parentenv, handler):
"No data available for site 0317154954"


...

GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=03351072&format=waterml,1.1&ParameterCd=00060&startDT=2014-01-01&endDT=2025-05-08



[1] "943/1056 - Downloading data for site 03352162"

GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=03352162&format=waterml,1.1&ParameterCd=00060&startDT=2014-01-01&endDT=2025-05-08



[1] "944/1056 - Downloading data for site 03353200"

GET: https://nwis.waterservices.usgs.gov/nwis/iv/?site=03353200&format=waterml,1.1&ParameterCd=00060&startDT=2014-01-01&endDT=2025-05-08

